In [ ]:
# Ensure you activated the Docker. See README.md for instructions. 
library(zellkonverter)
library(Matrix)
library(SingleCellExperiment)
library(dplyr)
library(tidyverse)
library(scalop)

# Annotate your dataset with the four GB states
- Compute the state scores
- Plot the quadrant
- Annotate in function of max score

### Load the data

In [ ]:
file_path <- "../data/adata.h5ad"

# Read the h5ad file into an R object
sce <- readH5AD(file_path)

# Inspect the loaded data
sce

### Compute the GB state scores for each cell

In [ ]:
# Extract the expression matrix (assuming 'X' is the matrix in the 'assays' slot)
m <- as.matrix(assay(sce, "logcounts"))

# Load the signature file (assuming it is saved as 'sigs.tsv')
sigs <- Signatures_GBM
sigs <- sigs[!(names(sigs) %in% c('G1S', 'G2M'))]


# Run sigScores with the specified parameters
sig_scores_result <- sigScores(
  m = m,            # The filtered expression matrix
  sigs = sigs,      # The gene signatures from the TSV file
  groups = NULL,    # Assuming no grouping for now
  center.rows = TRUE,   # Centering rows
  center = TRUE,        # Centering
  expr.center = TRUE,   # Expression center
  expr.bin.m = NULL,    # Not providing binning matrix
  expr.bins = NULL,     # No binning data
  expr.sigs = NULL,     # No binning signals
  expr.nbin = 30,       # Number of bins
  expr.binsize = 100,   # Size of each bin
  conserved.genes = 0.7, # Proportion of conserved genes
  replace = FALSE       # Do not replace existing data
)

# You can check the result
print(sig_scores_result)

# Save the result
saveRDS(sig_scores_result, file = "../results/sig_scores_result.rds")

In [ ]:
scores <- readRDS("../results/sig_scores_result.rds")

In [ ]:
scores <- as_four_state_gbm(scores)

In [ ]:
scores

### Plot quadrant

In [ ]:
# Plot quadrant
dat_hrchy = hierarchy(scores, quadrants=c('AC', 'MES', 'OPC', 'NPC'))
plot(dat_hrchy$X, dat_hrchy$Y,
     main = "Neftel from GBmap. Could reproduce quadrant",   # Title of the plot
     xlab = "X Values",                     # Label for the x-axis
     ylab = "Y Values",                     # Label for the y-axis
     pch = 19,                              # Type of point (filled circle)
     col = "blue")                          # Color of the points

### Annotate the cells

In [ ]:
# Annotate the cells in function of highest score. Choose min and diff in function of the scores in your dataset. 
# In our case, about 70% of the cells had a score above 0.3. 
annotation<-maxcol_sctrict(scores, min=0.3, diff=0.1)

In [ ]:
annotation